In [1]:
# Asyncronous Example & Simulation

import time
from abc import ABC, abstractmethod

class Task(ABC):
    @abstractmethod
    def run(): pass

class Sleep(Task):
    def __init__(self, duration:int = None):
        self.threshold = time.time() + duration
        self.ready = False    
    def run(self):
        now = time.time()
        if now >= self.threshold:
            self.ready = True

In [2]:
from typing import Iterable, Set, List

def wait(ts: Iterable[Task]):
    orig:List[Task] = list(ts)
    pending:Set[Task] = set(orig)
    before = time.time()
    
    while pending:
        for task in list(pending):
            task.run()
            if task.ready:
                pending.remove(task)
                
    print(f'duration = {time.time() - before}')

In [3]:
wait([Sleep(3) for _ in range(100)]) # the duration is same even thought in total there are hundred of sleep function

duration = 2.9999685287475586
